In [1]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

from pinecone import Pinecone
from dotenv import load_dotenv, find_dotenv
import os

import sys
sys.path.insert(0, '..')
from utility import get_GPT_response, client

import json


In [2]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.openai_config.env'))
api_key = os.environ.get('HACKATHON_API_KEY')

load_dotenv(os.path.join(os.path.expanduser('~'), '.pinecone_config.env'))
pinecone_api_key = os.environ.get('ANDREW_API_KEY')


In [3]:
embedding_model = "text-embedding-ada-002"

embed_model = OpenAIEmbedding(
    model=embedding_model,
    api_key=api_key,
)


Settings.embed_model = embed_model


In [4]:
pc = Pinecone(api_key=pinecone_api_key)

index_name = "eds"

pinecone_index = pc.Index(index_name)


In [5]:
query = "what drugs should be avoided by eds patients?"
top_k = 5
query_embedding = embed_model.get_text_embedding(
    query
)


In [6]:
retrieved_doc = pinecone_index.query(
    vector=query_embedding, 
    top_k=top_k, 
    include_metadata=True
)


In [14]:
for i in range(1):
    print(i)

0


In [71]:

extracted_context_summary = list(map(lambda x:json.loads(x.metadata['_node_content'])['metadata']['section_summary'], retrieved_doc.matches))
provenance = list(map(lambda x:x.metadata['c_document_id'], retrieved_doc.matches))


In [72]:

context = ''
for i in range(top_k):
    context += extracted_context_summary[i] + '(Ref: ' + provenance[i] + '. '


In [73]:
system_prompt = '''
You are an expert in the rare disease Ehlers-Danlos syndrome (EDS).
You are supposed to answer the question asked by the user. 
For this, you need to check the given context and then answer the question by grounding your response on the provided context.
If you are using the provided context, always make sure to provide the citation. You can find the citation in the context marked as 'Ref' 
'''

prompt = 'Context: ' + context + 'Query: ' + query

response = get_GPT_response(prompt, system_prompt, os.environ.get('MODEL_NAME'), temperature=0.3)



In [74]:
print(response)

Based on the provided context, individuals with Ehlers-Danlos Syndrome (EDS) should generally avoid long-term opioid use for pain management due to the associated risks and complications, such as tolerance, hyperalgesia, and potential overdose. The excerpt emphasizes the importance of cautious prescribing to avoid dependence, tolerance, and opioid use disorder in EDS patients (Ref: 10.7759/cureus.45713). Alternative pain management strategies, including non-opioid medications like NSAIDs, acetaminophen, anticonvulsants, and antidepressants, are recommended for long-term relief and improved function in individuals with EDS.
